<a href="https://colab.research.google.com/github/AhadAli451/PIAIC-Quarter-2-Projects/blob/main/02_Rag_Project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

###This is a first way to build embedding model with pinecone and through the langchain and add some like pinecone pineconeVectorStore

### 1 below write code to create embedding model
### 2 build a pineconevectorstore

In [ ]:
%pip install -qU  langchain-pinecone langchain-google-genai

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.3/41.3 kB 2.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 25.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 244.8/244.8 kB 14.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.4/85.4 kB 5.6 MB/s eta 0:00:00


In [ ]:
from google.colab import userdata

from pinecone import Pinecone, ServerlessSpec

pinecone_api_key = userdata.get('PINECONE_API_KEY')

pc = Pinecone(api_key=pinecone_api_key)

In [ ]:
index_name = "rag-project"  # change if desired

pc.create_index(
    name=index_name,
    dimension=768,
    metric="cosine",
    spec=ServerlessSpec(cloud="aws", region="us-east-1"),
)

index = pc.Index(index_name)

In [ ]:
from langchain_google_genai import GoogleGenerativeAIEmbeddings
import os

os.environ['GOOGLE_API_KEY'] = userdata.get('GOOGLE_API_KEY')

embeddings = GoogleGenerativeAIEmbeddings(model="models/embedding-001")

vector = embeddings.embed_query("how does work ai")
vector[:5]

[0.005915421061217785,
 -0.06183977797627449,
 -0.027067674323916435,
 -0.005238934885710478,
 -0.012123123742640018]

In [ ]:
from langchain_pinecone import pineconeVectorStore

vector_store = PineconeVectorStore(index=index, embedding=embeddings)

In [ ]:
from uuid import uuid4

from langchain_core.documents import Document

document_1 = Document(
    page_content="I had chocalate chip pancakes and scrambled eggs for breakfast this morning.",
    metadata={"source": "tweet"},
)

document_2 = Document(
    page_content="The weather forecast for tomorrow is cloudy and overcast, with a high of 62 degrees.",
    metadata={"source": "news"},
)

document_3 = Document(
    page_content="Building an exciting new project with LangChain - come check it out!",
    metadata={"source": "tweet"},
)

document_4 = Document(
    page_content="Robbers broke into the city bank and stole $1 million in cash.",
    metadata={"source": "news"},
)

document_5 = Document(
    page_content="Wow! That was an amazing movie. I can't wait to see it again.",
    metadata={"source": "tweet"},
)

document_6 = Document(
    page_content="Is the new iPhone worth the price? Read this review to find out.",
    metadata={"source": "website"},
)

document_7 = Document(
    page_content="The top 10 soccer players in the world right now.",
    metadata={"source": "website"},
)

document_8 = Document(
    page_content="LangGraph is the best framework for building stateful, agentic applications!",
    metadata={"source": "tweet"},
)

document_9 = Document(
    page_content="The stock market is down 500 points today due to fears of a recession.",
    metadata={"source": "news"},
)

document_10 = Document(
    page_content="I have a bad feeling I am going to get deleted :(",
    metadata={"source": "tweet"},
)

document_11 = Document(
    page_content="""AI systems work by combining large sets of data with intelligent,
     iterative processing algorithms to learn from patterns and features in the data
     that they analyze. Each time an AI system runs a round of data processing, it
      tests and measures its own performance and develops additional expertise. :(""",
    metadata={"source": "tweet"},
)

documents = [
    document_1,
    document_2,
    document_3,
    document_4,
    document_5,
    document_6,
    document_7,
    document_8,
    document_9,
    document_10,
    document_11,
]
uuids = [str(uuid4()) for _ in range(len(documents))]

vector_store.add_documents(documents=documents, ids=uuids)

['80f97987-509a-422b-bdb0-d2928ca6cb91',
 '56231b12-4dc7-40c8-ae27-5f059ca6a8e3',
 'a36d7d76-f08c-434f-8858-6661eeda4817',
 'f8f3433a-5384-4a45-895e-fc87410cc3de',
 '29b69054-9602-41ee-be10-7c84558f148a',
 '2bfbcb81-8180-496b-bbc6-d2144978de38',
 '3afcd0a5-dade-4609-b3da-7405d99ef09e',
 '15b7f232-dc14-4242-8ae4-94ae955b3fb9',
 '2ace1a1e-53c4-478f-b8fc-602cdd0587be',
 '4f9bfa66-4333-42a4-99da-5e4ac2ff416c',
 '17b44cc3-d407-4997-8a3d-ef5bc212c39f']

In [ ]:
len(documents)

11

In [ ]:
# Data Reterive

results = vector_store.similarity_search(
    "how does ai work",
    k=2,
    filter={"source": "tweet"},
)
for res in results:
    print(f"* {res.page_content} [{res.metadata}]")

# Data Reterive

* AI systems work by combining large sets of data with intelligent,
     iterative processing algorithms to learn from patterns and features in the data 
     that they analyze. Each time an AI system runs a round of data processing, it
      tests and measures its own performance and develops additional expertise. :( [{'source': 'tweet'}]
* Building an exciting new project with LangChain - come check it out! [{'source': 'tweet'}]


In [ ]:
results = vector_store.similarity_search_with_score(
    "how does work ai",k=2,filter={"source": "tweet"}
)
for res, score in results:
    print(f"* [SIM={score:3f}] {res.page_content} [{res.metadata}]")


* [SIM=0.655197] AI systems work by combining large sets of data with intelligent,
     iterative processing algorithms to learn from patterns and features in the data 
     that they analyze. Each time an AI system runs a round of data processing, it
      tests and measures its own performance and develops additional expertise. :( [{'source': 'tweet'}]
* [SIM=0.553384] Building an exciting new project with LangChain - come check it out! [{'source': 'tweet'}]


In [ ]:
from langchain_google_genai import ChatGoogleGenerativeAI

llm = ChatGoogleGenerativeAI(
    model="gemini-1.5-flash",
    temperature=0,
    max_tokens=None,
    timeout=None,
    max_retries=2,
    # other params...
)

In [ ]:
def answer_for_user(query: str):


  #Vector Serech
  vector_results = vector_store.similarity_search(query, k=2)
  print(len(vector_results))

  # TODO: pass to Model Vector Result + User Query
  final_answer = llm.invoke(f"This is a user query: {query}, Here are some references to answer{vector_results} ")


  return final_answer

In [ ]:
answer = answer_for_user("How does work ai")
answer.content

2


"Based on the provided text, AI systems work by:\n\n1. **Combining large datasets with intelligent algorithms:**  AI uses massive amounts of data as input.  This data is processed using sophisticated algorithms designed to identify patterns and features.\n\n2. **Iterative processing and learning:** The algorithms don't just process the data once. They repeatedly analyze the data, testing their performance and refining their understanding of the patterns within the data with each iteration.  This iterative process allows the AI to learn and improve its accuracy over time.\n\nThe second document is irrelevant to the question of *how* AI works.\n"

# Secound Work For Embedding

In [ ]:
# Pakistan zinda bad, we love our country.
# 0         1     2    3  4.   5.   6


In [ ]:
!pip install -q -U google-generativeai

In [ ]:
import google.generativeai as genai
from google.colab import userdata



genai.configure(api_key=userdata.get('GOOGLE_API_KEY'))

In [ ]:
list(genai.list_models())


[Model(name='models/chat-bison-001',
       base_model_id='',
       version='001',
       display_name='PaLM 2 Chat (Legacy)',
       description='A legacy text-only model optimized for chat conversations',
       input_token_limit=4096,
       output_token_limit=1024,
       supported_generation_methods=['generateMessage', 'countMessageTokens'],
       temperature=0.25,
       max_temperature=None,
       top_p=0.95,
       top_k=40),
 Model(name='models/text-bison-001',
       base_model_id='',
       version='001',
       display_name='PaLM 2 (Legacy)',
       description='A legacy model that understands text and generates text as an output',
       input_token_limit=8196,
       output_token_limit=1024,
       supported_generation_methods=['generateText', 'countTextTokens', 'createTunedTextModel'],
       temperature=0.7,
       max_temperature=None,
       top_p=0.95,
       top_k=40),
 Model(name='models/embedding-gecko-001',
       base_model_id='',
       version='001',
      

In [ ]:

from typing import Dict

result : Dict = genai.embed_content(
    model="models/text-embedding-004",
    content="What is the meaning of life?",
    task_type="retrieval_document",
    title="Embedding of single string",
)

# # 1 input > 1 vector output
# print(str(result["embedding"])[:50], "... TRIMMED]")

result['embedding']

[-0.02854543,
 0.044588115,
 -0.034197364,
 -0.0042663575,
 -0.04079577,
 0.012999958,
 0.018053582,
 0.06015144,
 -0.0028713925,
 0.009951648,
 0.024832657,
 -0.01683923,
 0.09940116,
 -0.031990346,
 0.018328529,
 -0.109134205,
 0.001190296,
 0.0014311911,
 -0.083155245,
 -0.010203233,
 0.019211812,
 0.0010217889,
 0.053874534,
 -0.0150861535,
 -0.003189089,
 0.019626662,
 -0.0074312133,
 -0.036586244,
 -0.008509182,
 -0.017352631,
 0.058202818,
 0.05446324,
 0.01571296,
 -0.021822602,
 0.048009068,
 0.022641798,
 -0.0069730366,
 0.054272633,
 0.025922865,
 -0.027334303,
 -0.07256842,
 0.028509492,
 -0.03564165,
 0.060492564,
 -0.022731686,
 -0.030770157,
 -0.006176277,
 -0.021891864,
 -0.019659325,
 0.0643669,
 0.03154234,
 0.017379418,
 -0.03679774,
 0.016511764,
 -0.02536976,
 -0.022270117,
 -0.012396498,
 -0.032805424,
 0.054154944,
 -0.04823156,
 -0.021759441,
 -0.03370158,
 -0.025460402,
 -0.017531719,
 -0.052902102,
 0.04005264,
 -0.022417234,
 0.023286799,
 -0.081740536,
 0.05

In [ ]:
len(result['embedding'])

768

In [ ]:
from typing import Dict

result : Dict = genai.embed_content(
    model="models/text-embedding-004",
    content=[
        "What is the meaning of life?",
        "How much wood would a woodchuck chuck?",
        "How does the brain work?",
    ],
    task_type="retrieval_document",
    title="Embedding of list of strings",
)

# A list of inputs > A list of vectors output
for v in result["embedding"]:
    print(str(v)[:50], "... TRIMMED ...", len(v))

[-0.036453027, 0.033254996, -0.03970925, -0.002628 ... TRIMMED ... 768
[-0.01591948, 0.032582663, -0.081024624, -0.011298 ... TRIMMED ... 768
[0.00037063024, 0.03763057, -0.122695684, -0.00951 ... TRIMMED ... 768


# Building Vector Stores & Retreival using Chroma DB

In [ ]:
!pip install -Uq langchain-chroma

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 3.6 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 628.3/628.3 kB 13.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 53.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 94.8/94.8 kB 5.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 278.6/278.6 kB 16.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 49.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 93.2/93.2 kB 5.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.3/13.3 MB 82.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.8/55.8 kB 3.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.9/54.9 kB 2.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 73.2/73.2 kB 3.9 MB/s eta 0:00:00


In [ ]:
import getpass
import os

In [ ]:
from langchain_core.documents import Document

documents = [
    Document(
        page_content="Dogs are great companions, known for their loyalty and friendliness.",
        metadata={"source": "mammal-pets-doc"},
    ),
    Document(
        page_content="Cats are independent pets that often enjoy their own space.",
        metadata={"source": "mammal-pets-doc"},
    ),
    Document(
        page_content="Goldfish are popular pets for beginners, requiring relatively simple care.",
        metadata={"source": "fish-pets-doc"},
    ),
    Document(
        page_content="Parrots are intelligent birds capable of mimicking human speech.",
        metadata={"source": "bird-pets-doc"},
    ),
    Document(
        page_content="Rabbits are social animals that need plenty of space to hop around.",
        metadata={"source": "mammal-pets-doc"},
    ),
]

In [ ]:
!pip install -Uq langchain- google-genai

ERROR: Invalid requirement: 'langchain-': Expected end or semicolon (after name and no valid version specifier)
    langchain-
             ^
